# Python 101
## Part XII.
---

## Time series

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BASE = './data/'

---
## Act I: Read historical USD-GBP-EUR currency rate data

Read data from the xlsx file.

We might have to install `openpyxl` lib first, with:
```bash
pip install openpyxl
```

In [ ]:
df = pd.read_excel(io=BASE+'currency_rates.xlsx', sheet_name='data')
df.head()

Set `Date` column as index (also sort the column).

In [ ]:
df = df.set_index('Date').sort_index()

In [ ]:
df.head()

#### Get the basic statistics about the data.

- Minimum

In [ ]:
df.min()

- Maximum

In [ ]:
df.max()

- Mean

In [ ]:
df.mean()

- Get every descriptive stat at once

In [ ]:
df.describe()

#### Let's focus on closing values!

In [ ]:
currencies = ['EUR/USD', 'GBP/USD', 'EUR/GBP']

def curr_list(suffix):
    return [currency + suffix for currency in currencies]

In [ ]:
df[curr_list(' Close')].plot();

In [ ]:
sns.boxplot(data=df[curr_list(' Close')]);

Create a new column called 'diff' which is the difference between the opening and the closing prices. Plot it!

In [ ]:
for currency in currencies:
    df[currency + '_diff'] = df[currency +' High'] - df[currency + ' Low'] 

df[curr_list('_diff')].plot();

Let's have a closer look at GBP at June! (Brexit consequencies)

In [ ]:
df.loc['2016-06-20':'2016-07-01', curr_list(' Close')].plot();

In [ ]:
sns.boxplot(data=df.loc['2016-06':'2016-07', curr_list('_diff')]);

In [ ]:
sns.boxplot(data=df.loc['2016-06':'2016-07', curr_list(' Close')]);

---
## Act II: Advanced operations

Let's focus on GBP, and save the closing price to a separate `Series`!

In [ ]:
# [c]loseing [p]rice [f]rame
cpf = df['EUR/GBP Close']
cpf

### Subact: Moving statistics

The most common operation is the moving average (rolling mean). Feel free to experiment with the window size!

In [ ]:
mavg = cpf.rolling(window=40).mean()
mavg.tail()

We plot the last 5 because:

In [ ]:
mavg.head(5)

Why?

---

After the answer, plot the moving average! 

In [ ]:
mavg.plot();

Put the two plot into the same axis system.

In [ ]:
cpf.plot(label='Close')
mavg.plot(label='Close AVG')
plt.legend();

We can apply several other types of functions with moving window, the complete list can be found [here](http://pandas.pydata.org/pandas-docs/stable/computation.html#window-functions).  

---

### Subact: Shifted computations

How can we compute something based on yesterday's data?  
Use the `shift` function!  
Let's compute the difference between the days:
$${d_t} = {p_t} - {p_{t-1}}$$

In [ ]:
df['gbp_diff'] = cpf - cpf.shift(1)
df['gbp_diff'].head()

In [ ]:
df['gbp_diff'].describe()

In [ ]:
df['gbp_diff'].plot();

In [ ]:
df.loc[df['gbp_diff'] > 0, 'EUR/GBP Close'].describe()

---
## Act III: Resampling

You can resample data to different time frequency. Two main parameters for the `resample` method is the time period you resemple to and the method that you use. By default the method is mean. The list time frames is accessable from [here](http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases).   

In [ ]:
# Monthly mean
cpf.resample('M').mean().plot();

In [ ]:
# Weekly median
cpf.resample('W').median().plot();

In [ ]:
# 3 weekly minimum
cpf.resample('3W').min().plot();

In [ ]:
# quarter year max
cpf.resample('QE').max().plot();

---

## Let's do some...

<img align="left" width=150 src="pics/magic.gif">
<br style="clear:left;"/>

### Act IV: Cool library of the week: <a href="https://https://streamlit.io/#install">Streamlit</a>

#### Create interactive web apps easily

**Streamlit** is one of the fastest ways to build interactive data apps in Python — without needing to know HTML, CSS, or JavaScript.
You write normal Python code, and Streamlit turns it into a **web app** with widgets, charts, text input, file upload, and more.

First, we write the contents of the demo streamlit app (you can write this manually):

In [ ]:
%%writefile moving_average_demo.py
import pandas as pd
import streamlit as st


# -------------------------------------------------------
#  Load data
# -------------------------------------------------------
df = (
    pd.read_excel(io='./data/currency_rates.xlsx', sheet_name='data')
    [['Date', 'EUR/GBP Close']]
    .sort_values("Date")
)


# -------------------------------------------------------
#  Streamlit UI
# -------------------------------------------------------
st.title("Moving Average Demo 📈")

st.write(
    "Use the slider to adjust the moving average window size "
    "and see how it affects the time series."
)

# Slider for window size
window = st.slider(
    "Moving average window size",
    min_value=1,
    max_value=60,
    value=10
)


# -------------------------------------------------------
#  Compute moving average
# -------------------------------------------------------
df["moving_average"] = df["EUR/GBP Close"].rolling(window).mean()


# -------------------------------------------------------
#  Visualization
# -------------------------------------------------------
st.line_chart(
    data=df, 
    x='Date',
    y=['EUR/GBP Close', 'moving_average'],
    color=['#008AD8', '#FEDD00']
)

st.write("### Raw Data")
st.dataframe(df)

Run your web app locally (you can also run this from the command line):

In [ ]:
!streamlit run moving_average_demo.py –server.headless true –browser.gatherUsageStats false

---
## Final Act: your play time!
### It's your turn - write the missing code snippets!

#### 1. Plot the changing of the EUR USD Closing values! Focus on the time of Trump's election! (2016-11-08)

#### 2. Which currency "won" the most on the election of Trump?

#### 3. Compute the moving sum on the "growing" days over a 30 day window (when the EUR/USD currency moved up)!  

In [ ]:
# 1. filter -> eur/usd > 0

# 2. rolling window = 30 + sum

# 3. plot


#### 4.a. Compute the moving standard deviation function
$${stddev} = \sqrt{\frac{\sum{({x}-\bar{x})^2}}{N}}$$
on the EUR/USD Closing values with 60 window size! 

#### 4.b. Plot it! 
Plot the original and the resampled (quarterly, mean) EUR/USD closing values into the same coordinate system (use the `style='--g'` argument in the `plot` method)!